<a href="https://colab.research.google.com/github/vatsal-kr/LegalNER-CS60075/blob/main/Legal_NER_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 24.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd
import json
import re
import glob
import nltk
import spacy
from sklearn.model_selection import train_test_split, KFold
from simpletransformers.ner.ner_model import NERModel
nltk.download('punkt')
# spacy.cli.download("en_core_web_md")
# nlp = spacy.load("en_core_web_md")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
judgement_data_train = pd.read_pickle('/content/drive/My Drive/LegalEval/judgement_data_train.pkl')
preamble_data_train = pd.read_pickle('/content/drive/My Drive/LegalEval/preamble_data_train.pkl')
judgement_data_dev = pd.read_pickle('/content/drive/My Drive/LegalEval/judgement_data_dev.pkl')
preamble_data_dev = pd.read_pickle('/content/drive/My Drive/LegalEval/preamble_data_dev.pkl')

In [ ]:
def get_df(judgement_data, preamble_data):
  all_tokens = pd.concat([judgement_data['text_tokenized'], preamble_data['text_tokenized']]).tolist()
  all_labels = pd.concat([judgement_data['label'], preamble_data['label']]).tolist()
  data = pd.DataFrame([])
  for i, (t, l) in enumerate(zip(all_tokens, all_labels)):
    assert len(t)==len(l)
    data = pd.concat([data, pd.DataFrame({
        'sentence_id': [i]*len(t),
        'words': t,
        'labels': l,
    })])
  return data

In [ ]:
train_data, test_data = get_df(judgement_data_train, preamble_data_train), get_df(judgement_data_dev, preamble_data_dev)

In [ ]:
with open('/content/drive/My Drive/LegalEval/tags.txt') as f:
  custom_labels = f.read().splitlines()

In [ ]:
def run_model(train_data, test_data, model_name, model_type, custom_labels):
  model = NERModel(model_type, model_name, labels = custom_labels)
  model.train_model(train_data, args={'num_train_epochs': 3, 'overwrite_output_dir': True,
                                      'learning_rate': 5e-5, 'warmup_ratio': 0.0, 'fp16': False,
                                      'manual_seed': 42})
  train_result, _, train_preds = model.eval_model(train_data)
  result, _, predictions = model.eval_model(test_data)

  return train_result, train_preds, result, predictions

In [ ]:
models = {
    # 'law-ai/InLegalBERT': 'bert',
    # 'nlpaueb/legal-bert-base-uncased': 'bert',
    # 'nlpaueb/bert-base-uncased-contracts': 'bert',
    # 'law-ai/InCaseLawBERT': 'bert',
    # 'zlucia/legalbert': 'bert',
    # 'pile-of-law/legalbert-large-1.7M-2': 'bert',
    'bert-base-uncased': 'bert',
    # 'saibo/legal-roberta-base': 'roberta',
    # 'roberta-base': 'roberta'
    # 'SpectaclesLLC/distilbert-legal-chunk': 'distilbert',
}

In [ ]:
def format_results(results_dct):
  
  models = results_dct['model_name']
  train_results = results_dct['train_results']
  test_results = results_dct['test_results']
  for m, tr_r, r in zip(models, train_results, test_results):
    print(f'------------------------MODEL NAME: {m}------------------------\nRESULTS ON TRAINING SET\n{tr_r}\n\n\nRESULTS ON DEV SET\n{r}\n\n')

In [ ]:
results_dct = {
    'train_results': [],
    'train_predictions': [],
    'test_results': [],
    'test_predictions': [],
    'model_name': []
}
tr_results, tr_preds, results, preds, model = [], [], [], [], []
for model_name, model_type in models.items():
  r1, p1, r2, p2 = run_model(train_data, test_data, model_name, model_type, custom_labels)
  results_dct['train_results'].append(r1)
  results_dct['train_predictions'].append(p1)
  results_dct['test_results'].append(r2)
  results_dct['test_predictions'].append(p2)
  results_dct['model_name'].append(model_name)

In [ ]:
format_results(results_dct)

------------------------MODEL NAME: law-ai/InLegalBERT------------------------
RESULTS ON TRAINING SET
{'eval_loss': 0.04682599759385498, 'precision': 0.8438, 'recall': 0.8759654513744706, 'f1_score': 0.8595819241269712}


RESULTS ON DEV SET
{'eval_loss': 0.1670679633954057, 'precision': 0.6504672897196262, 'recall': 0.7007652033830044, 'f1_score': 0.6746801085692128}


------------------------MODEL NAME: nlpaueb/legal-bert-base-uncased------------------------
RESULTS ON TRAINING SET
{'eval_loss': 0.048097694645212456, 'precision': 0.8435843265436281, 'recall': 0.8787891371148576, 'f1_score': 0.8608269438060567}


RESULTS ON DEV SET
{'eval_loss': 0.17569029923114513, 'precision': 0.6497175141242938, 'recall': 0.6947241240434958, 'f1_score': 0.6714674970805762}


------------------------MODEL NAME: nlpaueb/bert-base-uncased-contracts------------------------
RESULTS ON TRAINING SET
{'eval_loss': 0.05469375654652884, 'precision': 0.8136746725754758, 'recall': 0.8454201611577044, 'f1_score

## E-NER (Not part of baselines)

In [ ]:
!git clone https://github.com/terenceau2/E-NER-Dataset.git

Cloning into 'E-NER-Dataset'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 31 (delta 13), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), 3.19 MiB | 3.16 MiB/s, done.


In [ ]:
df = pd.read_csv('E-NER-Dataset/all.csv', header=None, names=['words', 'labels'])
df = df.fillna('')
df['labels'] = df['labels'].apply(lambda x: 'O' if x=='P' else x)
df['labels'] = df['labels'].apply(lambda x: 'I-LOCATION' if x=='I-LOC' else x)

In [ ]:
indices = df[df['words']=='.'].index

In [ ]:
sentences, labels = [], []
words, corr_labels = [], []
for i, row in df.iterrows():
  words.append(row['words'])
  corr_labels.append(row['labels'])
  if i in indices:
    sentences.append(words)
    labels.append(corr_labels)
    words = []
    corr_labels = []

In [ ]:
assert len(sentences)==len(labels)
for x, y in zip(sentences, labels):
  assert len(x)==len(y)

In [ ]:
df_new = pd.DataFrame([])
for i, (s, l) in enumerate(zip(sentences, labels)):
  df_new = pd.concat([df_new, pd.DataFrame([s, l, [i]*len(l)]).T])

In [ ]:
df_new.columns = ['words', 'labels', 'sentence_id']

In [ ]:
# train_data, test_data = train_test_split(df_new)

In [ ]:
label_list = list(df_new['labels'].unique())

In [ ]:
# results_dct = {
#     'train_results': [],
#     'train_predictions': [],
#     'test_results': [],
#     'test_predictions': [],
#     'model_name': []
# }
# for model_name, model_type in models.items():
#   r1, p1, r2, p2 = run_model(model_name, model_type, label_list)
#   results_dct['train_results'].append(r1)
#   results_dct['train_predictions'].append(p1)
#   results_dct['test_results'].append(r2)
#   results_dct['test_predictions'].append(p2)
#   results_dct['model_name'].append(model_name)

In [ ]:
format_results(results_dct)

In [ ]:
'''                     K-Fold Cross Validation                    '''

kf = KFold(n_splits=5, shuffle=True)
results_dct = {
      'train_results': [],
      'train_predictions': [],
      'test_results': [],
      'test_predictions': [],
      'model_name': []
  }
  
for model_name, model_type in models.items():  
  train_results, train_predictions, test_results, test_predictions = [], [], [], []
  for train_index, test_index in kf.split(df_new):
    train_data = df_new.iloc[train_index]
    test_data = df_new.iloc[test_index]
    
    r1, p1, r2, p2 = run_model(train_data, test_data, model_name, model_type, label_list)
    train_results.append(r1['f1_score'])
    train_predictions.append(p1)
    test_results.append(r2['f1_score'])
    test_predictions.append(p2)

  results_dct['train_results'].append(np.average(train_results))
  results_dct['train_predictions'].append(train_predictions)
  results_dct['test_results'].append(np.average(test_results))
  results_dct['test_predictions'].append(test_predictions)
  results_dct['model_name'].append(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/1260 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/1260 [00:00<?, ?it/s]

...

Running Epoch 2 of 3:   0%|          | 0/1260 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1260 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1237 [00:00<?, ?it/s]

In [ ]:
format_results(results_dct)

------------------------MODEL NAME: bert-base-uncased------------------------
RESULTS ON TRAINING SET
0.8224908486390966


RESULTS ON DEV SET
0.1999045751736431




In [ ]:
epoch1 = NERModel('bert', 'outputs/checkpoint-1260-epoch-1', labels = label_list)
epoch2 = NERModel('bert', 'outputs/checkpoint-2520-epoch-2', labels = label_list)
epoch3 = NERModel('bert', 'outputs/checkpoint-3780-epoch-3', labels = label_list)
ckpt2k = NERModel('bert', 'outputs/checkpoint-2000', labels = label_list)

In [ ]:
kf = KFold(n_splits=5, shuffle=True)
def cv_eval(model):
  test_results, test_predictions = [], []
  for train_index, test_index in kf.split(df_new):
      train_data = df_new.iloc[train_index]
      test_data = df_new.iloc[test_index]
      r, _, p = model.eval_model(test_data)
      
      test_results.append(r['f1_score'])
      test_predictions.append(p)
  return np.average(test_results)

In [ ]:
cv_eval(epoch1) # 0.1876186329962455

In [ ]:
cv_eval(epoch2)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1242 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1239 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1237 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1237 [00:00<?, ?it/s]

0.20546598903887198

In [ ]:
cv_eval(epoch3)

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1239 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1240 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1239 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1238 [00:00<?, ?it/s]

0.21295588741956845

In [ ]:
cv_eval(ckpt2k)

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1241 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1239 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1238 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1237 [00:00<?, ?it/s]

0.20611789445691997